In [1]:
# def checkResp(Res,rq_ResFn,rq_ResSFn):
#     Res = Res.decode()
#     Res_Fn = Res[0]+Res[1]
#     Res_SFn = Res[3]+Res[4]
# a= 8
# len(str(a))
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import pandas as pd
timeanalysis= pd.DataFrame([])
def encrypt_AES(Cryptkey, data):
    print(data)
    print(len(data))
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(key, encrypted_data):
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

# Example usage
# key = get_random_bytes(16)  # 16 bytes key for AES-128
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'
# data = b"Hello, world! This is AES encryption example."

# encrypted_data = encrypt_AES(key, data)
# print("Encrypted data:", encrypted_data)

# decrypted_data = decrypt_AES(key, encrypted_data)
# print("Decrypted data:", decrypted_data.decode())

In [2]:
# print(decrypt_AES(Cryptkey,encrypt_AES(Cryptkey,Cryptkey)))

In [3]:
def KeyGen(seed):
    global Cryptkey
    return encrypt_AES(Cryptkey, seed).hex()
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def ExtractKS (from_server, secAccLvl):
#     print(from_server[0:2])

    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 0: 
        print(from_server)
        print('ECU unlocked sucessfully')
        return 1
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 1:
        return '27 ' + ValLevel(str(secAccLvl+1)) + ' '+KeyGen(from_server[6:len(from_server)])
    return -1
# print(ExtractKS('67 01 67 68 69'))

In [4]:
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8080))
closeCon = 0
client.send("Tester Connected\n".encode())
secAccLvl = '07'
while closeCon ==0 :
    from_server = client.recv(4096)
    if from_server.decode() == 'End':
        closeCon = 1
        break
    print (from_server.decode())
    if int(secAccLvl)%2 == 1:
        resp = '27 '+ secAccLvl
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        print (from_server)
    resp = ExtractKS(from_server,int(secAccLvl))
    if resp == -1:
        print('error in message Fn or SubFn')
        closeCon = 1
        client.send(resp.encode())
        break
    elif resp == 1:
        print("ECU Unlocked")
        closeCon = 1
        client.send(resp.encode())
        break
    else:
        secAccLvl = str(int(secAccLvl)+1)
        closeCon = 1
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        resp = ExtractKS(from_server,int(secAccLvl))
client.close()



ECU Connected
27 07
67 07 c94a9174
c94a9174
8
27 08 22be2b611e716464ead09e85bd35d3ea34de97f8693fcc91cd3a6cf49b37a13f
67 08
ECU unlocked sucessfully


In [5]:
import socket
from datetime import datetime
for _ in range(0,100):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.1', 8080))
    closeCon = 0
    client.send("Tester Connected\n".encode())
    secAccLvl = '07'
    global timeanalysis
    start = datetime.now()
    while closeCon ==0 :
        from_server = client.recv(4096)
        if from_server.decode() == 'End':
            closeCon = 1
            break
        print (from_server.decode())
        if int(secAccLvl)%2 == 1:
            resp = '27 '+ secAccLvl
            print(resp)
            client.send(resp.encode())
            from_server = client.recv(4096).decode()
            print (from_server)
        resp = ExtractKS(from_server,int(secAccLvl))
        if resp == -1:
            print('error in message Fn or SubFn')
            closeCon = 1
            client.send(resp.encode())
            break
        elif resp == 1:
            print("ECU Unlocked")
            closeCon = 1
            client.send(resp.encode())
            break
        else:
            secAccLvl = str(int(secAccLvl)+1)
            closeCon = 1
            print(resp)
            client.send(resp.encode())
            from_server = client.recv(4096).decode()
            resp = ExtractKS(from_server,int(secAccLvl))
    client.close()
    end = datetime.now()
    elapsetime = (end - start).total_seconds()
    if elapsetime != 0 :
        timeanalysis = timeanalysis.append([elapsetime])

ECU Connected
27 07
67 07 f7416e92
f7416e92
8
27 08 869f326cd35b0d2f5430cdb9834405f5f664cb413a1a457458c13cd5590c0599
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 234440af
234440af
8
27 08 e4e06a747098d4c58c7701234fe5c9332695054408d61d5891dac34b9be906c7
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 cddfa298
cddfa298
8
27 08 fbce02a9d2b249eefc020e7f5e51cb053500214fa1163298f38b18e19591bd1a
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 c125253c
c125253c
8
27 08 9656bde7eee63443d0229f26c2c5efe44679fd69085b3ff0536c918cfe8059f9
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 b751b5df
b751b5df
8
27 08 6293d7bb97ee6c3836fb628192ea086ced4a1f3414ce7fcce4efbca1a8601d40
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 1e2cd88c
1e2cd88c
8
27 08 698598ba7344cfdd5cbdd0659ac6365951c38de152400c181b392fbd28b949b8
67 08
ECU unlocked sucessfully
ECU Connected
27 07
67 07 97186644
97186644
8
27 08 e4b5bc9df34969e8a588d6efe137df75ad2ec0055afded282aa70d7773bb

C:\Users\A477967\AppData\Local\Temp/ipykernel_29076/2841780769.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeanalysis = timeanalysis.append([elapsetime])
C:\Users\A477967\AppData\Local\Temp/ipykernel_29076/2841780769.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeanalysis = timeanalysis.append([elapsetime])
C:\Users\A477967\AppData\Local\Temp/ipykernel_29076/2841780769.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeanalysis = timeanalysis.append([elapsetime])
C:\Users\A477967\AppData\Local\Temp/ipykernel_29076/2841780769.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeanalysis = timeanalysis.appen

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [7]:
print('maximum number of iteration ',timeanalysis.values.max())
print('minimum number of iteration',timeanalysis.values.min()) 
print('Average number of iteration', timeanalysis.values.mean())

maximum number of iteration  0.002014
minimum number of iteration 0.000902
Average number of iteration 0.00143795


In [ ]:
from_server = '213456789'
print(from_server[5:9])

In [ ]:
# ExtractKS('67 01 51 52 54')
# def ExtractKS (from_server):
#     if from_server[0:1]  == '67' and int(from_server[3:4])%2 == 0: 
#         return from_server[5:len(from_server)]
#     if from_server[0:1]  == '27' and int(from_server[3:4])%2 == 1:
#         return from_server[5:len(from_server)]
#     return 0